In [ ]:
# Import required libraries/code
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import shutil
import os
import requests
import zipfile
import torch.backends.cudnn as cudnn
import pandas as pd

from pathlib import Path

from torch import nn
from torchvision import transforms, datasets

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# See if torchmetrics exists, if not, install it
try:
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >= 19, "mlxtend verison should be 0.19.0 or higher"
except:
    !pip install -q torchmetrics -U mlxtend # <- Note: If you're using Google Colab, this may require restarting the runtime
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    
# Import mlxtend upgraded version
import mlxtend 
print(mlxtend.__version__)
assert int(mlxtend.__version__.split(".")[1]) >= 19 # should be version 0.19.0 or higher

from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

from pathlib import Path

from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt
import numpy as np
import itertools

import numpy as np
from sklearn import metrics
import torchvision
import matplotlib.pyplot as plt

import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

try:
    import wandb
except:
    !pip install wandb boto3
    !pip install ipywidgets
    !apt-get update
    !apt-get install htop
    import wandb
    import boto3

import wandb    
import boto3
from botocore.exceptions import NoCredentialsError
import json
import random

try:
    import timm
except:
    !pip install timm
    import timm
    
from timm.scheduler.cosine_lr import CosineLRScheduler
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler

from timm.scheduler.step_lr import StepLRScheduler

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

In [ ]:
AWS_ACCESS_KEY = 'x'
AWS_SECRET_KEY = 'x'
region_name = 'us-east-1'
WANDB_API_KEY = 'x'
phone = 'x'

debug = False

SEED=1
ARCH = 'efficientnet_b5'

DROPOUT=0.2
EPOCHS = 30
LR = 0.01
decay_t = 5
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
NUM_CLASSES = 2
TRAIN_BATCH = 7
VAL_BATCH=TRAIN_BATCH
NUM_WORKERS = os.cpu_count()
wb_project = "RSNA"
model_save_file = "model_DNN.pth.tar"
s3_bucket = 'pparkitnrsna'
Scheduler_name = 'ReduceLROnPlateau' # 'CosineAnnealingLR' 'StepLR' 'ReduceLROnPlateau' 'MultiStepLR'
use_autocast = True
freeze_all_layers = True

# LOAD SAVED MODEL
LoadModel = False
LoadModelName = '/kaggle/input/rsna-wheel/model_best_playful-cloud-263.pth.tar'
if LoadModel == False:
    LoadModelName = "N/A"

# MODEL PARAMETERS

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

random.seed(SEED)
torch.manual_seed(SEED)

if ARCH == 'efficientnet_b7':
    imagesize = 600
    in_features=2560
    model_0 = torchvision.models.efficientnet_b7(pretrained=True,num_clases = NUM_CLASSES,weights='DEFAULT',in_chans=3).to(device)

elif ARCH == 'efficientnet_b5':
    imagesize = 448
    in_features=2048
    model_0 = torchvision.models.efficientnet_b5(pretrained=True,num_clases = NUM_CLASSES,weights='DEFAULT',in_chans=3).to(device)

elif ARCH == 'efficientnet_b2':
    imagesize   = 256
    in_features = 1408
    model_0 = torchvision.models.efficientnet_b2(pretrained=True,num_clases = NUM_CLASSES,weights='DEFAULT',in_chans=3).to(device)

elif ARCH == 'efficientnet_b0':
    imagesize   = 244
    in_features =1280
    model_0 = torchvision.models.efficientnet_b0(pretrained=True,num_clases = 1,weights='DEFAULT',in_chans=3).to(device)

# SETUP W&B

In [ ]:
os.environ["WANDB_API_KEY"] = WANDB_API_KEY

wandb.login()
run  = wandb.init(project=wb_project, entity='pparkitny', config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, 
                   "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH, "imagesize":imagesize, "NUM_CLASSES":NUM_CLASSES})

wandb_run_name = wandb.run.name
wandb_run_id = wandb.run.id

config = wandb.config
config.learning_rate = LR
config.Scheduler_name = Scheduler_name
config.LoadModel = LoadModel
config.LoadModelName = LoadModelName
config.decay_t = decay_t
config.use_autocast = use_autocast
config.DROPOUT = DROPOUT
config.debug=debug
config.freeze_all_layers = freeze_all_layers

wandb.run.log_code(".")

### Get data 

In [ ]:
# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
    # Download pizza, steak, sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...") 
        zip_ref.extractall(image_path)

    # Remove .zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

# Setup Dirs
train_dir = image_path / "train"
test_dir = image_path / "test"

In [ ]:
!rm data/pizza_steak_sushi/test/pizza/ -R
!rm data/pizza_steak_sushi/train/pizza/ -R

In [ ]:
if debug == False:       
    train_dir = '/kaggle/input/rsna-1024-tt-dup-a/train/tmp/working/data/1024/train'
    test_dir = '/kaggle/input/rsna-1024-tt-dup-a/test/tmp/working/data/1024/test'

### Prepare data

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((imagesize, imagesize)), 
    #transforms.RandomAffine(degrees=3),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(degrees=(-5, 5)),    
    #transforms.RandomPerspective(distortion_scale=0.8, p=1.0),
    #transforms.RandomAffine(degrees=(-10, 10), translate=(0.1, 0.3), scale=(0.9, 0.99)),    
    #transforms.RandomEqualize(),
    #transforms.RandomAutocontrast(),
    #transforms.RandomSolarize(threshold=192.0),
    #transforms.RandomPosterize(bits=2),
    #transforms.RandomInvert(),
    #transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.Grayscale(num_output_channels=3), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) 
])

test_transform = transforms.Compose([
    transforms.Resize((imagesize, imagesize)), 
    transforms.Grayscale(num_output_channels=3), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) 
])

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)

class_names = train_data.classes

train_dataloader = DataLoader(
      train_data,
      batch_size=TRAIN_BATCH,
      shuffle=True,
      num_workers=NUM_WORKERS,
      pin_memory=True,
  )

test_dataloader = DataLoader(
      test_data,
      batch_size=TRAIN_BATCH,
      shuffle=False,
      num_workers=NUM_WORKERS,
      pin_memory=True,
  )

print(len(train_dataloader.dataset))
print(class_names)

# PF1 Score

In [ ]:
def pfbeta(labels, predictions, beta):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

### Get and prepare a pretrained model

In [ ]:
if freeze_all_layers == True:

    # Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
    for param in model_0.features.parameters():
        param.requires_grad = False
        
if 1==2:
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 4:
            for param in child.parameters():
                param.requires_grad = False

In [ ]:
summary(model_0)

In [ ]:
summary(model=model_0, input_size=(TRAIN_BATCH, 3, imagesize, imagesize),col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"],
        verbose=2,
        depth=1
       )

In [ ]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
model_0.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=DROPOUT, inplace=True), 
    torch.nn.Linear(in_features=in_features,
                    out_features=output_shape, 
                    bias=True)).to(device)


In [ ]:
summary(model_0)

# LOAD SAVED MODEL

In [ ]:
if LoadModel == True:
    print("Loading Model",LoadModelName)
    
    # Setup device agnostic code
    device = "cuda" if torch.cuda.is_available() else "cpu"
    device

    if torch.cuda.is_available() == True:
        loc = 'cuda:{}'.format('0')
        model_0 = torch.load(LoadModelName)
        cudnn.deterministic = True
        cudnn.benchmark = True
    else:
        loc=torch.device('cpu') 
        model_0 = torch.load(LoadModelName,map_location=torch.device('cpu'))

### Train model

In [ ]:
def upload_to_aws(local_file, bucket, s3_file):
    s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY,
                      aws_secret_access_key=AWS_SECRET_KEY)
    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

def download_from_aws(s3_file, bucket):
    s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY,
                      aws_secret_access_key=AWS_SECRET_KEY)
    try:
        s3.download_file(bucket, s3_file, s3_file)
        print("Download Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False
    
def send_sms(phone,text,aws_access_key_id,aws_secret_access_key,region_name):
    # Create an SNS client
    client = boto3.client(
        "sns",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name
    )

    # Send sms message.
    client.publish(
        PhoneNumber=phone,
        Message=text
    )
    
def list_files_from_aws(bucket):
    fileList = []
    s3 = boto3.resource('s3', aws_access_key_id=AWS_ACCESS_KEY,aws_secret_access_key=AWS_SECRET_KEY)
    my_bucket = s3.Bucket(bucket)
    for my_bucket_object in my_bucket.objects.all():
        fileList.append(str(my_bucket_object.key))
    return fileList

def filter_list(input_list,contains):
    
    list_return = []
    
    for item in input_list:
        if item.find(contains) != -1:
            list_return.append(item)
    
    return(list_return)

def remove_from_list(input_list,contains):
    
    list_return = []
    
    for item in input_list:
        if item.find(contains) == -1:
            list_return.append(item)
    
    return(list_return)

In [ ]:
def save_checkpoint(state, is_best, filename_in,model):
    torch.save(state, filename_in)
    torch.save(model,'model'+filename_in)
    
    if is_best:
        shutil.copyfile(filename_in, 'weights_best_'+wandb_run_name+'.pth.tar')
        upload_to_aws('weights_best_'+wandb_run_name+'.pth.tar', s3_bucket, 'weights_best_'+wandb_run_name+'.pth.tar')
        
        shutil.copyfile('model'+filename_in, 'model_best_'+wandb_run_name+'.pth.tar')
        upload_to_aws('model_best_'+wandb_run_name+'.pth.tar', s3_bucket, 'model_best_'+wandb_run_name+'.pth.tar')

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
        
        #LOG WANDB
        #wandb.log({"Loss/train": loss, 'acc1/train': train_acc })

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:

    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:

    best_acc1 = 0
    best_epoch = 0
    
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        
        lr_rate=get_lr(optimizer)
        
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"LR: {lr_rate:.6f} | "  
          f"train_loss: {train_loss:.6f} | "
          f"train_acc: {train_acc:.6f} | "
          f"test_loss: {test_loss:.6f} | "
          f"test_acc: {test_acc:.6f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        # remember best acc@1 and save checkpoint
        is_best = test_acc > best_acc1
        best_acc1 = max(test_acc, best_acc1)
        
        if is_best == True:
            best_epoch = epoch
                
        PATH = str(epoch) + "_" +str(test_acc) + "_" + ARCH + ".tar" 
        save_checkpoint({
        'epoch': epoch,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),}, is_best,PATH,model)
        
        scheduler.step(epoch)
                
        wandb.log({'lr':lr_rate, 
                   'epoch':epoch,
                   "best_acc1/val": best_acc1,
                   "best_epoch": best_epoch,
                  'train_loss':train_loss,
                  'train_acc':train_acc,
                  'test_loss':test_loss,
                  'test_acc':test_acc})    
        

    # Return the filled results at the end of the epochs
    return results

In [ ]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr=LR)
scheduler = timm.scheduler.StepLRScheduler(optimizer, decay_t = decay_t, decay_rate=.5)

In [ ]:
%%time

# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Setup training and save the results
model_0_results = train(model=model_0,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=EPOCHS,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# LOAD BEST MODEL WEIGHTS

In [ ]:
model_name = 'weights_best_'+wandb_run_name+'.pth.tar'

if torch.cuda.is_available() == True:
    loc = 'cuda:{}'.format('0')
else:
    loc=torch.device('cpu') 

checkpoint = torch.load(model_name, map_location=loc)
print(checkpoint['arch'])
model_0.load_state_dict(checkpoint['state_dict'])

In [ ]:
class_names

In [ ]:
class_names = test_data.classes
class_names

# LOAD BEST FULL MODEL

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((imagesize, imagesize)), 
    transforms.Grayscale(num_output_channels=3), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) 
])

test_data = datasets.ImageFolder(test_dir, transform=test_transform)

test_dataloader = DataLoader(
      test_data,
      batch_size=TRAIN_BATCH,
      shuffle=False,
      num_workers=NUM_WORKERS,
      pin_memory=True,
  )

class_names = test_data.classes
print("class_names",class_names)

model_name = 'model_best_'+wandb_run_name+'.pth.tar'

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

if torch.cuda.is_available() == True:
    loc = 'cuda:{}'.format('0')
    model_0 = torch.load(model_name)
else:
    loc=torch.device('cpu') 
    model_0 = torch.load(model_name,map_location=torch.device('cpu'))
    
#put model in evaluation mode
model_0.eval()

### Make predictions on the entire test dataset with the model

In [ ]:
# Make predictions on the entire test dataset
test_preds = []
test_probs = []
model_0.eval()
with torch.inference_mode():
  # Loop through the batches in the test dataloader
  for X, y in tqdm(test_dataloader):

    X, y = X.to(device), y.to(device)
    # Pass the data through the model
    test_logits = model_0(X)

    # Convert the pred logits to pred probs
    pred_probs = torch.softmax(test_logits, dim=1)

    # Convert the pred probs into pred labels
    pred_labels = torch.argmax(pred_probs, dim=1)

    # Add the pred labels to test preds list
    test_preds.append(pred_labels)
    test_probs.append(pred_probs)

# Concatenate the test preds and put them on the CPU
test_preds = torch.cat(test_preds).cpu()

test_probs = torch.cat(test_probs).cpu()

In [ ]:
test_probs

# MAKE ROC GRAPH

In [ ]:
def make_ROC_graph(labels_test,prediction):
    """ Text """

    false_positive_rate, recall, thresholds = roc_curve(labels_test,prediction)
    roc_auc = auc(false_positive_rate, recall)
    fig = plt.figure(figsize=(3, 3),frameon =False, dpi=200)  
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, recall, 'b', label='AUC = %0.2f' %roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    fn = "roc_graph.png"
    plt.savefig(fn,bbox_inches='tight')
    plt.show()
    plt.close()
    wandb.log({"Media/ROC-Graph": wandb.Image("roc_graph.png")})
    config.AUC = roc_auc
    return roc_auc

## PREDICTIONS

In [ ]:
classes = class_names
classes

In [ ]:
if debug == False:
    test_data_paths = list(Path(test_dir).glob("*/*.png"))
else:
    test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
test_data_paths[:5]

In [ ]:
# Get all test data paths
test_labels = [path.parent.stem for path in test_data_paths]

# Create a function to return a list of dictionaries with sample, label, prediction, pred prob
def pred_and_store(test_paths, model, transform, class_names, device):
  test_pred_list = []
  for path in tqdm(test_paths):
    # Create empty dict to store info for each sample
    pred_dict = {}

    # Get sample path
    pred_dict["image_path"] = path

    # Get class name
    class_name = path.parent.stem
    pred_dict["class_name"] = class_name

    # Get prediction and prediction probability
    from PIL import Image
    img = Image.open(path) # open image
    transformed_image = transform(img).unsqueeze(0) # transform image and add batch dimension
    model.eval()
    with torch.inference_mode():
        pred_logit = model(transformed_image.to(device))
        pred_prob = torch.softmax(pred_logit, dim=1)
        pred_label = torch.argmax(pred_prob, dim=1)
        pred_class = class_names[pred_label.cpu()]

        # Make sure things in the dictionary are back on the CPU 
        pred_dict["pred_prob"] = pred_prob.unsqueeze(0).max().cpu().item()
        pred_dict["pred_class"] = pred_class
                        
        pred_dict["pred_label_int"] = classes.index(pred_class)
        pred_dict["class_name_int"] = classes.index(class_name)
        
  
    # Does the pred match the true label?
    pred_dict["correct"] = class_name == pred_class

    # print(pred_dict)
    # Add the dictionary to the list of preds
    test_pred_list.append(pred_dict)

  return test_pred_list

test_pred_dicts = pred_and_store(test_paths=test_data_paths,
                                 model=model_0,
                                 transform=test_transform,
                                 class_names=class_names,
                                 device=device)

test_pred_dicts[:1]

In [ ]:
actual_name = []
pred_class = []
pred_logit = []
pred_label = []
pred_label_int = []
class_name_int = []

for rec in test_pred_dicts:
    actual_name.append(rec['class_name'])
    class_name_int.append(rec['class_name_int'])
    pred_class.append(rec['pred_class'])
    pred_label_int.append(rec['pred_label_int'])
    
make_ROC_graph(class_name_int,pred_label_int)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig = plt.figure(figsize=(4, 4),frameon =False, dpi=200)  
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cnf_matrix = confusion_matrix(actual_name, pred_class)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,title='Confusion matrix')
plt.savefig("confusion-matrix.png",bbox_inches='tight')
plt.show()
wandb.log({"Media/Confusion Matrix": wandb.Image("confusion-matrix.png")})

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes, normalize=True,title='Normalized confusion matrix')
plt.savefig("confusion-matrix-normalized.png",bbox_inches='tight')
plt.show()
wandb.log({"Media/Normalized Confusion Matrix": wandb.Image("confusion-matrix-normalized.png")})

print(classification_report(actual_name, pred_class, target_names=classes))
print(accuracy_score(actual_name, pred_class))

config.classification_report = classification_report(actual_name, pred_class, target_names=class_names)

In [ ]:
# Turn the test_pred_dicts into a DataFrame

test_pred_df = pd.DataFrame(test_pred_dicts)

# Sort DataFrame by correct then by pred_prob 
top_5_most_wrong = test_pred_df.sort_values(by=["correct", "pred_prob"], ascending=[True, False]).head()
top_5_most_wrong

In [ ]:
# Plot the top 5 most wrong images
for row in top_5_most_wrong.iterrows():
    row = row[1]
    image_path = row[0]
    true_label = row[1]
    pred_prob = row[2]
    pred_class = row[3]
    # Plot the image and various details
    img = torchvision.io.read_image(str(image_path)) # get image as tensor
    plt.figure()
    plt.imshow(img.permute(1, 2, 0)) # matplotlib likes images in [height, width, color_channels]
    plt.title(f"True: {true_label} | Pred: {pred_class} | Prob: {pred_prob:.3f}")
    plt.axis(False);

In [ ]:
# Make a function to pred and plot images
def pred_and_plot(image_path, model, transform, class_names, device=device):
  # open image
  image = Image.open(image_path)

  # transform image
  transformed_image = transform(image)

  # pred on image
  model.eval()
  with torch.inference_mode():
    pred_logit = model(transformed_image.unsqueeze(0).to(device))
    pred_label = torch.argmax(torch.softmax(pred_logit, dim=1), dim=1)
  
  # plot image and pred
  plt.figure() 
  plt.imshow(image)
  plt.title(f"Pred: {class_names[pred_label]}")
  plt.axis(False);

In [ ]:
# Try again on a photo of steak from unsplash.com 
!wget https://images.unsplash.com/photo-1546964124-0cce460f38ef
!cp photo-1546964124-0cce460f38ef steak.jpg

pred_and_plot("steak.jpg",
              model=model_0,
              transform=test_transform,
              class_names=class_names)

In [ ]:
# Plot loss curves of a model
def plot_loss_curves(results):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    loss = results["train_loss"]
    test_loss = results["test_loss"]

    accuracy = results["train_acc"]
    test_accuracy = results["test_acc"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

    plt.savefig("loss_curves.png",bbox_inches='tight') 
    plt.show()

In [ ]:
# Plot the loss curves of our model
plot_loss_curves(model_0_results)
wandb.log({"Media/Loss_Curves": wandb.Image("loss_curves.png")})

# CALCULATE PFBETA

In [ ]:
#class_name_int,pred_label_int

labels = class_name_int
predictions = pred_label_int
beta = 1
pfbeta_val = pfbeta(labels, predictions, beta)
config.pfbeta=pfbeta_val
print(pfbeta_val)

In [ ]:
sms_text = wandb_run_name + " Is Complete " 
#send_sms("+16479807463",sms_text,AWS_ACCESS_KEY,AWS_SECRET_KEY,region_name)

# Mark the run as finished
wandb.finish()